In [27]:
!pip install --quiet dtlpy

In [28]:
import json
import argparse
from pathlib import Path
import os.path
import dtlpy as dataloop
import subprocess
from PIL import Image
import urllib.request 
import random
import numpy as np

In [58]:
data_info = {
    'project': "Car Condition Classification",
    'dataset': "all_portrait",
    'email': "ashley.lawrencehuizenga@coxautoinc.com",
    'out_dir': "dataloop/",
    'password': ""
}

In [59]:
dataloop.login_m2m(email=data_info['email'], password=data_info['password'])

True

In [31]:
proj = dataloop.projects.get(data_info['project'])

In [32]:
dataset = proj.datasets.get(data_info['dataset'])

In [10]:
dataset.download(local_path=data_info["out_dir"])
dataset.download_annotations(local_path=data_info["out_dir"])

Download Items:   0%|          | 0/1 [00:00<?, ?it/s]
Download Item:   0%|          | 0.00/40.8M [00:00<?, ?B/s]
Download Item:   1%|▏         | 568k/40.8M [00:00<00:12, 3.39MB/s]
Download Item:   4%|▎         | 1.46M/40.8M [00:00<00:06, 6.15MB/s]
Download Item:   6%|▌         | 2.46M/40.8M [00:00<00:05, 7.85MB/s]
Download Item:  14%|█▍        | 5.69M/40.8M [00:00<00:02, 17.2MB/s]
Download Item:  24%|██▍       | 9.80M/40.8M [00:00<00:01, 26.0MB/s]
Download Item:  34%|███▍      | 14.0M/40.8M [00:00<00:00, 31.7MB/s]
Download Item:  45%|████▍     | 18.3M/40.8M [00:00<00:00, 35.8MB/s]
Download Item:  55%|█████▌    | 22.5M/40.8M [00:00<00:00, 38.6MB/s]
Download Item:  65%|██████▌   | 26.6M/40.8M [00:00<00:00, 39.7MB/s]
Download Item:  75%|███████▌  | 30.8M/40.8M [00:01<00:00, 41.0MB/s]
Download Item:  86%|████████▌ | 35.0M/40.8M [00:01<00:00, 41.9MB/s]
Download Item: 100%|██████████| 40.8M/40.8M [00:01<00:00, 32.6MB/s]
Download Items: 100%|██████████| 1/1 [00:01<00:00,  1.44s/it]


'dataloop/'

In [33]:
pages = dataset.items.list()

In [34]:
seen_filenames = set()
all_names = {}

In [37]:
for page in pages:
    for item in page:
        all_names[item.id] = item.name

Iterate Pages: 100%|██████████| 66/66 [00:58<00:00,  1.13it/s]


In [45]:
for name in all_names:
    filename = all_names[name]

In [46]:
filename

'-aXVFYcBUMhyNhiE4ueA_4.jpg'

In [47]:
random_file_list = set()

In [51]:
for name in all_names:
    filename = all_names[name]
    if "_" in filename:
        unique_filename = filename[: filename.rfind("_")]
    else:
        unique_filename = filename
    if unique_filename not in seen_filenames:
        seen_filenames.add(unique_filename)
        files_with_prefix = [
            f
            for f in all_names
            if all_names[f].startswith(unique_filename)
        ]
        random_file_list.add(random.choice(files_with_prefix))

In [53]:
dataset.items.get(item_id = files_with_prefix[0])

Item(dataset_url='https://gate.dataloop.ai/api/v1/datasets/65552b994f2e91b053fdd998', created_at='2023-11-15T20:36:54.132Z', updated_at='2023-11-15T20:38:31.581Z', updated_by='rubiks@dataloop.ai', dataset_id='65552b994f2e91b053fdd998', filename='/539F54YB-YxIUEnRGvhi_2.jpg', name='539F54YB-YxIUEnRGvhi_2.jpg', type='file', id='65552c47ee8a97b2fad0eedc', spec=None, creator='ashley.lawrencehuizenga@coxautoinc.com', _description=None, annotations_count=1)

In [54]:
files = []
labels = []
file_names = []
class_to_label = {"clean": 0, "light_dirt": 1, "heavy_dirt": 1}

for id in random_file_list:
    item = dataset.items.get(item_id = id)
    try:
        item.download(local_path="temp/")
        target_file = os.path.join('temp', item.name)
        
        item.annotations.download(filepath="temp/")
        annotation_file_name = os.path.join('temp', os.path.splitext(item.name)[0] + ".json")
        
        with open(annotation_file_name) as annotation_file:
            json_data = json.load(annotation_file)
            img_width = json_data["metadata"]["system"]["width"]
            img_height = json_data["metadata"]["system"]["height"]
            annotations = []
            for annotation in json_data["annotations"]:
                label = annotation["label"]
                try:
                    coords = annotation["coordinates"]
                    p1 = np.array([coords[0]["x"], coords[0]["y"]])
                    p2 = np.array([coords[1]["x"], coords[1]["y"]])
                    bbox_size = np.linalg.norm(p2 - p1)
                    annotations.append(
                        [bbox_size, class_to_label[label], coords]
                    )
                except KeyError:
                    try:
                        annotations.append([1, class_to_label[label], 0])
                    except KeyError:
                        continue
            annotations = sorted(annotations, key=lambda x: x[0], reverse=True)
            # Only the biggest bounding box is considered
            if len(annotations) == 0:
                continue
            files.append(target_file)
            labels.append(annotations[0])
            file_names.append(item.name)
    except FileNotFoundError:
                # We just ignore files that are not images or do not have a json file
                continue

Download Items: 100%|██████████| 1/1 [00:00<00:00,  3.94it/s]


In [18]:
files[5]

'test/-0ss7IYBUMhyNhiERWaY_8.jpg'

In [55]:
len(files) == len(labels)

True